In [1]:
from pyincore import IncoreClient
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil



In [2]:
client = IncoreClient()


In [3]:
building_dmg = "5c12856d1f5e0d0667d6410e"
housing_unit_alloc = "5d543b06b9219c0689b987af"
bg_data = "5d542bd8b9219c0689b90408"


In [4]:
pop_dis = PopulationDislocation(client)


In [5]:
pop_dis.load_remote_input_dataset("building_dmg", building_dmg)
pop_dis.load_remote_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)

Dataset already exists locally. Reading from local cache.
Dataset already exists locally. Reading from local cache.
Dataset already exists locally. Reading from local cache.


In [6]:
#pop_dis.show_gdocstr_docs()

result_name = "housing-dislocation-result"
seed = 1111

pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)



True

In [7]:
pop_dis.run_analysis()

True